In [1]:
from fastai import * 
from fastai.text import * 
import numpy as np
import io
import pathlib
from fastai_sentencepiece import *
from filetokenizer import *
from languagemodelloader import *

In [3]:
lang="en"
minToks = 10

In [42]:
pathData       = Path("../../data/nlp")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"

pathTrainValid = path/"wiki-train_valid"
pathTxt        = pathTrainValid/"txt"
pathToks       = pathTrainValid/"toks"
pathcsv        = pathTrainValid/"wiki.csv"

cache_name   = "sp-model"
pathVocab    = pathTrainValid / cache_name

In [43]:
#Discard section with kess than "minTok" tokens
spt_func  = partial(SentencepieceTokenizer.create, pathVocab=pathVocab)
spt_func.__name__ = SentencepieceTokenizer.__name__
spt       = spt_func(lang="en")
pad_idx   = spt.vocab().numericalize([text.transform.PAD])[0]
vocab,max_vocab = spt.vocab(), len(spt.vocab().itos)
trainTokenizer = FileTokenizer(pathToks/"train", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))
validTokenizer = FileTokenizer(pathToks/"valid", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))

print(trainTokenizer)
print("size og vocabulary:", max_vocab)
print("pad_idx:",pad_idx)

print(spt.vocab().numericalize( ["xxunk" ,"xxbos","xxeos" ,"xxpad" ,"xxmaj" ,"xxup" ,"xxrep" ,"xxwrep", "xxfld"]  ))
#sentence = ["She is tall.", "He is small"]
#tokenizer._process_all_1(sentence)

self.dtype:<class 'numpy.int16'>
self.dtype:<class 'numpy.int16'>
Tokenizer SentencepieceTokenizer in en with the following rules:
 - fix_html
 - replace_rep
 - replace_wrep
 - spec_add_spaces
 - rm_useless_spaces
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - rm_extra_lineshift
 - replace_all_caps
 - deal_caps

size og vocabulary: 32000
pad_idx: 0
[0, 1, 2, 0, 3, 4, 5, 6, 0]


In [44]:
toks   =  np.asarray([v for v in spt.vocab().itos])
tokslen = np.asarray([len(v) for v in spt.vocab().itos])
ix_sort = np.argsort(tokslen)
#tokslen = tokslen[ix_sort]
#toks    = toks[ix_sort]
print("number og len(toks)=)1", np.sum(tokslen==1))

number og len(toks)=)1 5727


In [45]:
charaters = ",".join(toks[tokslen==1])
#np.sum(toks=='a')
charaters

',,.,s,-,\',",),▁,e,:,d,a,y,t,;,n,i,o,S,A,r,m,’,k,g,C,B,l,c,D,u,b,z,I,K,p,h,2,T,E,–,P,F,G,M,1,f,x,W,L,3,H,R,O,V,4,%,N,—,>,w,U,j,J,5,é,v,],6,8,X,Y,7,9,”,Z,0,(,á,&,Q,!,ā,q,í,<,ó,?,ä,+,ō,š,ü,_,ī,=,ö,ú,ū,*,ı,ł,°,ă,å,$,и,ç,č,à,è,ë,[,ø,ž,α,ş,о,а,е,|,â,ę,ê,æ,ș,т,ß,с,ñ,ô,ð,ا,ś,л,×,ν,ι,ρ,н,ο,р,τ,ą,ý,‘,ε,д,“,в,ा,ż,ğ,ř,ț,î,λ,É,σ,π,र,्,у,ć,−,ر,},κ,»,ē,ě,م,м,`,ò,ʻ,ن,ə,ي,ń,ő,γ,к,η,ل,ì,õ,г,ė,ς,و,đ,β,ï,′,μ,@,й,υ,δ,ω,ب,ù,я,்,~,د,п,б,י,Ü,ã,ы,θ,ि,·,ţ,क,ь,س,†,з,ו,‐,Ś,म,ч,त,न,ṣ,ί,ό,ạ,ר,±,С,व,ŏ,ả,ά,ṭ,û,स,ت,ी,Δ,Ú,ی,М,φ,ه,ư,े,œ,А,ה,χ,ḥ,ʿ,ệ,ع,ц,א,В,ल,य,द,中,Í,ح,า,^,Æ,Α,К,П,ु,Ö,ṇ,Ω,х,Б,⁄,大,→,þ,प,Σ,£,„,ב,ế,ة,έ,ं,ň,€,Þ,ف,ů,≤,ա,ร,Š,ồ,ל,ج,ź,ह,ύ,州,Ä,ق,Д,ش,ш,¥,Н,і,ή,ж,க,Т,מ,王,ị,•,Π,Г,ज,О,ộ,{,Р,Ș,ש,ो,ி,ṃ,子,ー,道,公,್,、,村,ு,山,ת,ர,Κ,ग,®,ŋ,த,ũ,́,Ć,श,ம,น,Ḥ,ћ,Ε,人,ξ,İ,ǎ,ע,ब,ŭ,ா,Γ,נ,ņ,ப,≥,天,ز,ك,ன,Č,ი,文,Á,Î,И,ḍ,ン,ờ,ಿ,¡,Μ,Φ,¢,Λ,כ,া,്,ĩ,ק,ј,李,ζ,ח,ḩ,भ,ъ,خ,ậ,の,镇,ட,∈,ა,Ž,э,国,≈,★,ว,פ,Ț,أ,₤,∞,ώ,家,三,平,เ,ṛ,Ñ,Л,च,ÿ,ד,อ,ย,ť,Ţ,ص,Τ,日,ấ,§,ಾ,À,ั,ắ,Ο,ม,南,ն,一,ס,ط,ง,司,ು,ச,«,・,ן,ನ,武,ː,ರ,У,ф,ם,ิ,ọ,щ,Ø,街,ľ,太,西,安,ю,ล,ψ,ố,江,生,্,ǔ,上,ण,บ,ề,金,東,國,Å,Ō,ʾ,₹,

In [46]:
spt.vocab().numericalize(["a","b","ab","xxbos"])

[56, 299, 1700, 1]

In [47]:
spt.vocab().textify([56,299])

'a b'

In [48]:
#np.asarray([65536],dtype=np.int16)
np.asarray([65536/2-1],dtype=np.int16)

array([32767], dtype=int16)

In [58]:
text="Oak Creek, Colorado\n\n\nOak Creek is a Statutory Town in Routt County, Colorado, United States. The population was 849 at the 2000 census. It was incorporated in 1907 as a coal mining town. The community was named for scrub oak near the original town site.\nOak Creek is located at (40.275049, -106.957607)."
text="[[Russian Revolution (1917)|Russian Revolution|Russian Revolution]]]"

text="in the historical canon), are the anarchist territories during the [[Spanish Revolution of 1936|Spanish Revolution]] and "+\
     "the [[Free Territory]] during the [[Russian Revolution (1917)|Russian Revolution]].Through the efforts "+\
     "and influence of the [[Anarchism in Spain|Spanish anarchists]] during the Spanish ,詹,誠,词,遇,邱,郵,釋,鎭,閑,"
text=text+text
#text=["\nin the historical canon),", "\nare the anarchist territories during the \n[[Spanish Revolution of 1936|Spanish Revolution]] and ",
#      "the [[Free Territory]] during the [[Russian Revolution (1917)|Russian Revolution]].\nThrough the efforts ","and influence of the [[Anarchism in Spain|Spanish anarchists]] during the Spanish\n\n\n"]

In [59]:
#import fastai_sentencepiece

def extract_link_title(t:str) -> str:
    return re.sub('\[\[([^\]^|\[:]+)|([^\]\[:]+)\]\]', '\g<1>', t).replace(']]','')
    #return re.sub('\[\[(?:([^\]\[:]+))|([^\]\[:]+)\]\]', '\g<1>', t).replace('[','')
    #return re.sub('\[\[(?:([^\]\[:]+))|([^\]\[:]+)\]\]', '\g<1>', t)
def remove_first_empty_lines(t:str) -> str:
    return re.sub('^\n', '', t)

def keep_western(t:str) -> str:
    return ''.join(re.findall('([A-Za-z\d_ -/:-@\[-`{-~])',t))
    #return  re.sub('\', '', t)
                   
#\p{IsLatin}|[! -/:-@\[-`{-~]|[0-9]
                         
print(text)
print("------------") 
#for i,t in enumerate(text):
#    print(extract_link_title(t))
#    print(remove_first_empty_lines(t))
#%timeit re.sub('\[\[([^\]^|\[:]+)|([^\]\[:]+)\]\]', '\g<1>', text)
print(keep_western(text))
print("------------")



in the historical canon), are the anarchist territories during the [[Spanish Revolution of 1936|Spanish Revolution]] and the [[Free Territory]] during the [[Russian Revolution (1917)|Russian Revolution]].Through the efforts and influence of the [[Anarchism in Spain|Spanish anarchists]] during the Spanish ,詹,誠,词,遇,邱,郵,釋,鎭,閑,in the historical canon), are the anarchist territories during the [[Spanish Revolution of 1936|Spanish Revolution]] and the [[Free Territory]] during the [[Russian Revolution (1917)|Russian Revolution]].Through the efforts and influence of the [[Anarchism in Spain|Spanish anarchists]] during the Spanish ,詹,誠,词,遇,邱,郵,釋,鎭,閑,
------------
in the historical canon), are the anarchist territories during the [[Spanish Revolution of 1936|Spanish Revolution]] and the [[Free Territory]] during the [[Russian Revolution (1917)|Russian Revolution]].Through the efforts and influence of the [[Anarchism in Spain|Spanish anarchists]] during the Spanish ,,,,,,,,,,in the historical ca

In [44]:
defaults.cpus

12

In [12]:
lang     = "en"
pathData = Path("../nlp-data")
path     = pathData / lang
pToks    = path/"wiki-train_valid"/"toks"


In [ ]:
import re
#p = re.compile('([\r\n]+.?)+', r'\r\n')
p = re.compile("[\r\n]+.?")
txt = " 1\n\n\n\n\n"
p.sub("\n",txt)

In [ ]:
imax = int(1e8)
def tuppel():
    for i in range(imax): 
        r = (i,i+1)
        l = r[1]-r[0]
%timeit tuppel()    

In [ ]:
def simpleParams():
    for i in range(imax): 
        r0 = i
        r1 = i+1
        l  = r1-r0
%timeit simpleParams()        

In [ ]:
lang     = "en"
pathData = Path("../nlp-data")
path     = pathData / lang
pTrain   = path/"wiki-train_valid"/"toks"/"train"

def loadAll(pTrain):
    for p in pTrain.glob("*.npy"):
        with p.open("rb") as f:
            np.load(f)
%time loadAll(pTrain)
#s=None
gc.collect()

In [ ]:
np.load??

In [ ]:
?np.load

In [ ]:
%%timeit
lang     = "en"
pathData = Path("../nlp-data")
path     = pathData / lang
p        = path/"wiki-train_valid"/"toks"/"train"/"0-ids.npy"
    
a=None
gc.collect()

In [ ]:
            import tracemalloc
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()

            -----your interesting code
            
            snapshot2 = tracemalloc.take_snapshot()
            top_stats = snapshot2.compare_to(snapshot1, 'lineno')
            print(f"Top 10 of {len(top_stats)}")
            for stat in top_stats[:10]: print(stat)
          

In [ ]:
data = buffer[:bs*3].reshape(bs,3)

In [ ]:
-1%9

In [ ]:
l=9
for i in range(2*l):print(f"i:{i} i-backwards:{l-1 - i%l}")

In [ ]:
seq_len=3
a = np.arange(bs*seq_len,-1,-1)
ix = np.arange(len(a))
print(ix)
print(a)
ts[ix] = a[ix]
print(t.shape)
print(t)

In [ ]:
t.numel()/3

In [ ]:
bs=4
sl=3
bs_dim=t.numel()/sl
t.view(-1,14)

In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

In [ ]:
import asyncio
async def myfun():
    await asyncio.sleep(1)
    return 5

In [ ]:
await myfun()
await myfun()


In [ ]:
async def hello():
    await asyncio.sleep(1)
    return "hello"
async def test():
    loop = asyncio.get_event_loop()
    tasks = []
    for i in range(10):
        tasks.append(loop.create_task(hello()))
    # all the tasks will automatically run
    asyncio.set_event_loop(loop)
    rest = asyncio.gather(*tasks)
    await rest

In [ ]:
#await test()

loop = asyncio.get_event_loop()
tasks = []
for i in range(10):
    tasks.append(loop.create_task(hello()))
# all the tasks will automatically run
asyncio.set_event_loop(loop)
rest = asyncio.gather(*tasks)
await rest

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool 

pool = ThreadPool(1) 
results = pool.map(test,None)
pool.close() 
pool.join()


In [ ]:
        """
        async def hello():
            await asyncio.sleep(1)
            return "hello"
    
        fileblocks = partition_by_cores(files, n_cpus)
        print(f"fileblocks:{len(fileblocks)}")
        tasks = []
        i=0
        loop = asyncio.get_event_loop()
        for fb in fileblocks:
            print(f"task:{i}")
            i += 1
            results = tasks.append( loop.create_task( FileTokenizer.getIds_from_file(fb) ) )
        asyncio.set_event_loop(loop)
        results = asyncio.gather(*tasks)
        await results
        """


In [ ]:
lang            = "en"
pathData       = Path("../nlp-data")
path           = pathData / lang
pathTrainValid = path/"wiki-train_valid"
pathNPY        = pathTrainValid/"dummy.npy"
pathNPZ        = pathTrainValid/"dummy.npz"


In [ ]:
arrays = np.empty(10, dtype=object)
arrays

In [ ]:
import io
a=np.arange(3,dtype=np.int32).astype(np.int32)
b=np.arange(5,dtype=np.int32).astype(np.int32)
c=np.arange(10,dtype=np.int32).astype(np.int32)


In [ ]:
with pathNPY.open("wb") as f:
    np.save(f,a, allow_pickle=False, fix_imports=False)
    np.save(f,b, allow_pickle=False, fix_imports=False)
    np.save(f,c, allow_pickle=False, fix_imports=False)

with pathNPY.open("rb") as f:
    f.seek(0, 2); file_size = f.tell(); f.seek(0)
    while f.tell() != file_size:
        print(np.load(f))


In [ ]:
arrays = np.empty(0, dtype=object)
arrays

In [ ]:
arrays = [] #np.empty(3, dtype=object)
for i in range(3):
    #for i,a in enumerate(arrays):
    arrays.append( np.arange(i+1) )

#arrays2 = np.empty(3, dtype=object)
#arrays2[:3] = arrays[:3]
#arrays[1][0] = 10
print(arrays)

def save(arrays):
    with pathNPY.open("wb") as f:
        if isinstance(arrays,list): 
            arrays = np.asarray(arrays,dtype=object)
        np.save(f,arrays, allow_pickle=True, fix_imports=False)
        
def load():
    with pathNPY.open("rb") as f:
        arrays = np.load(f)
    return arrays

save(arrays)
load()

In [ ]:
%timeit save(arrays)

In [ ]:
%timeit load()
print(load())

In [ ]:
arrays = [] #np.empty(0, dtype=object)
for i in range(3):
    a = load()
    if len(a) > 0: arrays.extend( a.tolist() )
arrays = np.asarray(arrays,dtype=object)    
print(len(arrays))
print(arrays)